In [16]:
# warning 무시
from warnings import filterwarnings
filterwarnings('ignore')

In [17]:
# glob : 파일 리스트 뽑기
from glob import glob

# tqdm : for문 진행률 확인
from tqdm import tqdm

from collections import deque

In [18]:
# requests : HTTP를 사용하기 위해 쓰여지는 라이브러리
import requests

# multiprocessing : process spawing
import multiprocessing
import pandas as pd
import numpy as np
import os

# 질문
1. flag : 수집되는 거 안되는 거 구분?
A : request 했는데 값이 안들어오는 id 기록하려고. result_error 

2. recursive = True 용도? 

3. df['num_lang'] = df['languages'].fillna('').apply(lambda x: len(x.split())) 결측치 공백 입력, x : df['languages'].fillna('') 인가요? 
A. language 가 없는 애들 None -> "" 으로 바꿔 해당 컬럼을 string으로 바꿈. 그리고 띄어쓰기로 구분된 언어를 리스트로 만들어 개수 세기

4. df.to_csv(f'{steamspy_dir}\\{file_name}.csv', index = False, sep = '\t')
    index = False?
A. df 를 저장하면 행 번호가 생김. index 행번호 안넣기
index = True 하면 Unnamed:0 이 컬럼으로 생기고, read_csv 할 때 

5. .ipynb 로컬에서 돌리고 .py 파일로 모으기. py 파일은 상위폴더를 가장 위를 불러오고 ipynb는 바로 위 폴더
6. 

In [19]:
cwd = os.getcwd() # 로컬에서 파일을 가져오려고 cwd -> 같은 위치에 있는 상대경로 활용
# c:\Users\NT550009\Desktop\ASAC_ml_project\web_crawling\JC\2_steampy_tag

check_path = glob(f'{cwd}\\**\\steamspy.txt', recursive = True)[0]
# C:\Users\NT550009\Desktop\ASAC_ml_project\web_crawling\JC\2_steampy_tag\steamspy.txt

In [34]:
def extract(appid):
    flag = ''
    cwd = os.getcwd()  # 로컬에서 파일을 가져오려고 cwd -> 같은 위치에 있는 상대경로 활용
    # steamspy.txt로 되있는 파일경로 찾아서 리스트로 출력은 ['steamspy.txt의 경로']
    check_path = glob(f'{cwd}\\**\\steamspy.txt', recursive=True)[0]  # \\**\\ 경로 설정을 편하게. 성현님 코드에서 txt와 ipynb 가 다른 곳에 있음.


    # steamspy.txt에서 모을 내용 리스트 원소로 정리
    # steampy.txt를 줄 기준으로 읽어 온뒤 \n(엔터키)를 제거 open(check_path).readlines()의 출력은 ['1.~~~~\n', '2.~~~~\n' .....] 이런식
    # 결과적으로 check_list = ['1.~~~~', '2.~~~~' .....]
    check_list = [i.replace('\n', '') for i in open(check_path).readlines()]

    # appid 당 check_list 원소 정리
    # result_dict = {appid: {'1.~~~~':'', '2.~~~~': '', ........}}
    result_dict = {appid: {i: '' for i in check_list}}

    # sample : https://steamspy.com/api.php?request=appdetails&appid=1794680
    res = requests.get(f'https://steamspy.com/api.php?request=appdetails&appid={appid}')
    
    # 요청이 성공적으로 수행되지 않았을 때
    if res.status_code != 200:  # status_code : 응답상태 확인
        flag = False
        return result_dict, flag

    # json 형식 dict으로 가져오기
    res_json = res.json()
    
    for check in check_list:
        # 예외 처리
        # res_json 에서 check 키에 내용 찾아서 추가
        # result_dict = {appid: {'1.~~~~': res_json[check], ........}}
        try:
            result_dict[appid][check] = res_json[check]
        except:
            continue

    flag = True

    return result_dict, flag
    



In [31]:
# steamspy_dir = glob(f'{cwd}',recursive=True)[0]
print(glob(f'{cwd}'))
steamspy_dir

['c:\\Users\\NT550009\\Desktop\\ASAC_ml_project\\web_crawling\\JC\\2_steampy_tag']


[]

In [33]:
def run():
    cwd = os.getcwd()
    pwd = os.path.abspath(os.path.join(cwd, '..'))
    # 1_appid라는 이름이 포함되있는 txt 파일경로를 리스트로 가져옴(recursive=True로 하고 ** 쓰면 하위폴더까지 전부 검색)
    appid_dirs = glob(f'{pwd}\\**\\1_appid\\*.txt', recursive=True)
    # steamspy.txt로 되있는 파일경로 찾아서 리스트로 출력은 ['steamspy.txt의 파일 경로'] (하위 폴더까지 전부 검색)
    check_path = glob(f'{cwd}\\**\\steamspy.txt', recursive=True)[0]
    
    # 저장 위치 지정
    steamspy_dir = glob(f'{cwd}', recursive=True)[0]

    # steampy.txt를 줄 기준으로 읽어 온뒤 \n(엔터키)를 제거 open(check_path).readlines()의 출력은 ['1.~~~~\n', '2.~~~~\n' .....] 이런식
    check_list = [i.replace('\n', '') for i in open(check_path).readlines()]

    for appid_dir in appid_dirs:
        # appid_dirs 리스트 안에 있는 appid_dir들의 파일 이름만 추출
        # os.path.basename(appid_dir) : 경로, 파일명과확장자 분리
        base_name = os.path.basename(appid_dir)

        # # 파일 이름 안에 2019 혹은 2018 이 있으면 제외
        # if '2019' in base_name or '2018' in base_name:  # 2018, 2019 pass
        #     continue

        # appid_dir.txt를 줄 기준으로 읽어 온뒤 \n(엔터키)를 제거 open(appid_dir).readlines()의 출력은 ['~~~~\n', '~~~~\n' .....] 이런식
        app_list = [i.replace('\n', '') for i in open(appid_dir).readlines()]

        # 딕셔너리 컴프리헨션
        result_last = {i: deque([]) for i in check_list}  # {'developer': deque([]), 'publisher': deque([]) } 형식
        result_last["appid"] = app_list

        result_mid = {}
        result_error = {}

        # app_list로 extract()함수 반복
        # for i in app_list:
        #     a = extract(i)
        #     print(a)
        
        # 병렬처리
        # with multiprocessing.Pool() as p: # 병렬처리 : 파이썬에서 원래 안되는데 나눠서 하고 모음
        #     for val, flag in tqdm(p.imap(extract, app_list, 8), total = len(app_list)):
        #         result_mid.update(val)
        #         if flag == False:
        #             result_error.update(val)
        
        # 직렬처리
        for appid in app_list:
            val, flag = extract(appid)
            
            # dict.update({} ) : dict에 dict 추가. 
            # A.update({B:C}) 랑 A[B] = C 같다
            result_mid.update(val)
            if flag == False:
                result_error.update(val)
        
        
        for app in app_list:
            for check in check_list:
                result_last[check].append(result_mid[app][check])

        df = pd.DataFrame(result_last)

        # 결측치 처리
        df['price'] = df['price'].fillna(0)
        df['price'] = df['price'].astype(int) / 100

        df['initialprice'] = df['initialprice'].fillna(0)
        df['initialprice'] = df['initialprice'].astype(int) / 100


        df['num_lang'] = df['languages'].fillna('').apply(lambda x: len(x.split()))
        
        # 파일명과확장자 분리
        file_name = os.path.splitext(base_name)[0]

        df.to_csv(f'{steamspy_dir}\\{file_name}.csv', index=False, sep='\t')

# cmd 창에 run() 결과만 보여주는 방법
# multiprocessing 쓸 때 필수
if __name__ == '__main__':
    run()


glob(f'{cwd}\\**\\steamspy.txt', recursive=True)[0]

완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완

'c:\\Users\\NT550009\\Desktop\\ASAC_ml_project\\web_crawling\\JC\\2_steampy_tag\\steamspy.txt'

In [ ]:
check_path = glob(f'{cwd}\\**\\steamspy.txt', recursive = True)[0]
check_list = [i.replace('\n','') for i in open(check_path).readlines()]
result_last= {i : deque([]) for i in check_list} 
print(result_last)

{'developer': deque([]), 'publisher': deque([]), 'positive': deque([]), 'negative': deque([]), 'average_forever': deque([]), 'average_2weeks': deque([]), 'median_forever': deque([]), 'median_2weeks': deque([]), 'price': deque([]), 'initialprice': deque([]), 'languages': deque([]), 'tags': deque([]), 'genre': deque([])}
